+ When:
    + 2021-08-14
+ What
	+ Profile some implementations of matrix inner product
+ Why
	+ Learn how to profile a python implementation
+ How
	+ 1. Implement some features of inner product of two matrices, m1 and m2
	+ 2. Run all the implementations with a same set of parameters
	+ 3. Compare elipsed times for the implemntations
	+ 4. Learn tips for inner product implementations

In [1]:
import cProfile
import numpy as np

# 1. Implement features
Implement a feature of inner product of two matrices, m1 and m2, in two ways

In [2]:
def f1(m1, m2):
    # m1: (n1, n2), m2: (n2, n3)
    return m1 @ m2

In [3]:
def f2(m1, m2):
    # m1: (n1, n2), m2: (n2, n3)
    return np.matmul(m1, m2)

In [4]:
def f3(m1, m2):
    # m1: (n1, n2), m2: (n2, n3)
    n1, n2 = m1.shape
    _, n3 = m2.shape
    m3 = np.zeros((n1, n3))
    for i in range(n1):
        for k in range(n3):
            for j in range(n2):
                m3[i,k] += m1[i,j] * m2[j,k]
    return m3

In [5]:
def f4(m1, m2):
    # m1: (n1, n2), m2: (n2, n3)
    n1, n2 = m1.shape
    _, n3 = m2.shape
    m2_ = m2.T
    m3 = np.zeros((n1, n3))
    for i in range(n1):
        for k in range(n3):
            for j in range(n2):
                m3[i,k] += m1[i,j] * m2_[k,j]
    return m3

In [6]:
def f5(m1, m2):
    # m1: (n1, n2), m2: (n2, n3)
    n1, n2 = m1.shape
    _, n3 = m2.shape
    m2_ = m2.T
    m3 = np.zeros((n1, n3))
    for i in range(n1):
        for k in range(n3):
            m3[i,k] = m1[i,:] @ m2[:,k]
    return m3

In [10]:
def f6(m1, m2):
    # m1: (n1, n2), m2: (n2, n3)
    n1, n2 = m1.shape
    _, n3 = m2.shape
    m2_ = m2.T
    m3 = np.zeros((n1, n3))
    for j in range(n2):
        m3 += m1[:,j,None] * m2[None,j,:]
    return m3

In [11]:
m1 = np.random.randn(10,2)
m2 = np.random.randn(2,3)

assert np.max(np.abs(f1(m1, m2) - f2(m1, m2))) < 1e-8
assert np.max(np.abs(f1(m1, m2) - f3(m1, m2))) < 1e-8
assert np.max(np.abs(f1(m1, m2) - f4(m1, m2))) < 1e-8
assert np.max(np.abs(f1(m1, m2) - f5(m1, m2))) < 1e-8
assert np.max(np.abs(f1(m1, m2) - f6(m1, m2))) < 1e-8

# 2. Implement a caller
Run the two methods with a same set of parameters

In [14]:
n1, n2, n3 = 2**5, 2**8, 2**5
m1, m2 = np.random.randn(n1, n2), np.random.randn(n2, n3)
def main(nRepeat):
    for _ in range(nRepeat):
        f1(m1, m2)
        f2(m1, m2)
        f3(m1, m2)
        f4(m1, m2)
        f5(m1, m2)
        f6(m1, m2)

# 3. Profile
Compare elipsed times for two implemntations

In [16]:
cProfile.run("main(32)", sort="tottime")

         324 function calls in 13.292 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       32    6.648    0.208    6.648    0.208 <ipython-input-4-8440f1c32040>:1(f3)
       32    6.543    0.204    6.543    0.204 <ipython-input-5-cd5086534581>:1(f4)
       32    0.058    0.002    0.058    0.002 <ipython-input-6-e0185ad864b1>:1(f5)
       32    0.040    0.001    0.040    0.001 <ipython-input-10-ad047bf171d5>:1(f6)
       32    0.001    0.000    0.001    0.000 <ipython-input-2-267aabae791c>:1(f1)
       32    0.001    0.000    0.001    0.000 <ipython-input-3-1d70a58175a5>:1(f2)
      128    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000   13.292   13.292 <ipython-input-14-0f0a4b22ee4a>:3(main)
        1    0.000    0.000   13.292   13.292 {built-in method builtins.exec}
        1    0.000    0.000   13.292   13.292 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000

# 4. Discussions
Learn which implemtation way is better

* When passing an inner product of a pair of matrices, it's highly reocommended to use matmul or the operator @.
* When being unable to avoid random access to matrix elements, minimize the number of the access like in f5 or f6.